In [1]:
import pandas as pd
import numpy as np
import re, html
from bs4 import BeautifulSoup as BS, NavigableString, SoupStrainer
from html_table_parser import parser_functions

In [2]:
import collections
if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

parser_d0350 = SoupStrainer("section-2")
section2_pattern = re.compile(r"<SECTION-2((?!<SECTION-2)[\S\s\n])*?(D-0-3-5-0)[\S\s\n]*?</SECTION-2>")

In [3]:
# 파일시스템에서 xml 문서들을 불러오기 

import os

path = 'C:/Users/Samsung/Documents/KPMG/kic/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xml')] 

path_line = []
for i in file_list_py:

    path= 'C:/Users/Samsung/Documents/KPMG/kic/'+ i
    path_line.append(path)

#print(path_line) 

In [103]:
df_base = pd.DataFrame(columns=['full_text'])

for i in range(len(path_line)):
    # 인코딩 방식이 다른 문헌의 경우 예외를 적용하도록 함.
    try : 
        with open(path_line[i], 'r', encoding="cp949") as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")
    except:
        with open(path_line[i], 'r', encoding="utf-8",errors='ignore') as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")

    # xml 문서에서 텍스트 형태로 변환하기  
    txt_line = [list(text.stripped_strings) for text in remark_page.find_all(recursive=False) if list(text.stripped_strings) != [] and 'border="1"' not in text.prettify()]
    raw_txt = sum(txt_line, [])
    txt = "\n".join(raw_txt)
    df_base = df_base.append(pd.DataFrame([txt], columns=['full_text']))


C:\Users\Samsung\AppData\Local\Temp\ipykernel_86824\511267560.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_base = df_base.append(pd.DataFrame([txt], columns=['full_text']))
C:\Users\Samsung\AppData\Local\Temp\ipykernel_86824\511267560.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_base = df_base.append(pd.DataFrame([txt], columns=['full_text']))
C:\Users\Samsung\AppData\Local\Temp\ipykernel_86824\511267560.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_base = df_base.append(pd.DataFrame([txt], columns=['full_text']))
C:\Users\Samsung\AppData\Local\Temp\ipykernel_86824\511267560.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

In [97]:
base_report = df_base.reset_index(inplace=False).drop(columns=['index'])
base_report.to_csv("base_report(\n).csv")

In [26]:
# 표 확인
table1 = remark_page.find('table', border=1)
pd.DataFrame(parser_functions.make2d(table1))

,0,1,2,3,4
0,주주,주식의 종류,주식수(주),자본금(천원),지분율(%)
1,대신자산신탁㈜,보통주,"400,000","2,000,000",18.71
2,기타,종류주,"1,738,000","8,690,000",81.29
3,합계,합계,"2,138,000","10,690,000",100


In [102]:
with open( 'C:/Users/Samsung/Documents/KPMG/kic/20210510000378.xml', 'r', encoding="cp949", errors='ignore') as data_xml:
    dsd_xml = data_xml.read()
    dsd_xml = dsd_xml.replace('&cr;', '&#13;')
    dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
    dsd_xml = html.unescape(dsd_xml)
    section2_section = section2_pattern.search(dsd_xml)
    section2_section = section2_section.group()
    
remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")

[pd.DataFrame(parser_functions.make2d(table)) for table in remark_page.find_all('table', border=1, recursive=False)]

[      0            1           2       3
 0  구  분          주주명      주식수(주)  지분율(%)
 1   보통주  미래에셋자산운용(주)   1,050,593    4.38
 2   보통주           기타  22,949,407   95.62
 3  합  계   24,000,000      100.00    None,
       0          1          2
 0  구  분        당기말        전기말
 1  보통예금  4,240,794  3,939,795,
       0     1          2    3
 0  구  분  금융기관        당기말  전기말
 1  정기예금  우리은행  3,000,000    -,
       0      1          2
 0  구  분    당기말        전기말
 1   미수금      -  1,798,411
 2  미수수익  6,198      1,574
 3  합  계  6,198  1,799,985,
       0      1      2
 0  구  분    당기말    전기말
 1  선급비용  2,825  1,961,
       0        1    2
 0  구  분      당기말  전기말
 1  선납세금  719,760  777,
                         0       1       2             3            4       5  \
 0                    구  분     소재지     지분율          출자좌수         기  초    취  득   
 1                  (수익증권)  (수익증권)  (수익증권)        (수익증권)       (수익증권)  (수익증권)   
 2  이지스전문투자형사모\r부동산투자신탁97호  대한\r민국  99.91%  113,200\r백만좌  148,818,380       -   
 

In [17]:
df_table = pd.DataFrame(columns=['full_text'])

for i in range(len(path_line)):
    # 인코딩 방식이 다른 문헌의 경우 예외를 적용하도록 함.
    try : 
        with open(path_line[i], 'r', encoding="cp949") as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")
    except:
        with open(path_line[i], 'r', encoding="utf-8",errors='ignore') as data_xml:
            dsd_xml = data_xml.read()
            dsd_xml = dsd_xml.replace('&cr;', '&#13;')
            dsd_xml = re.sub('(\n|\r)?</*SPAN.*?>(\n|\r)?', '', dsd_xml)
            dsd_xml = html.unescape(dsd_xml)
            section2_section = section2_pattern.search(dsd_xml)
            section2_section = section2_section.group()
        remark_page=BS(section2_section, 'lxml', parse_only=parser_d0350).find("section-2")

    # xml 문서에서 텍스트 형태로 변환하기  
    table_line = [pd.DataFrame(parser_functions.make2d(table)) for table in remark_page.find_all('table', border=1,recursive=False)]
    raw_txt = sum( txt_line, [])
    txt = "   ".join(raw_txt)
    df_base = df_base.append(pd.DataFrame([txt], columns=['full_text']))


[         0       1          2           3       4
 0       주주  주식의 종류     주식수(주)     자본금(천원)  지분율(%)
 1  대신자산신탁㈜     보통주    400,000   2,000,000   18.71
 2       기타     종류주  1,738,000   8,690,000   81.29
 3       합계      합계  2,138,000  10,690,000     100,
       0     1
 0  계정과목  내용연수
 1    건물   40년,
            0          1
 0  (단위 : 천원)  (단위 : 천원)
 1         구분         당기
 2       보통예금    358,930,
            0          1           2          3          4           5
 0  (단위 : 천원)  (단위 : 천원)   (단위 : 천원)  (단위 : 천원)  (단위 : 천원)   (단위 : 천원)
 1         과목         기초          취득         처분       감가상각          기말
 2         토지          -  15,707,670          -          -  15,707,670
 3         건물          -  19,734,243          -          -  19,734,243
 4    감가상각누계액          -           -          -  (202,772)   (202,772)
 5         합계          -  35,441,913          -  (202,772)  35,239,141,
           0         1         2           3           4           5
 0  (단위: 천원)  (단위: 천원)  (단위: 천